In [30]:
import json
import sys
import glob
import torch
sys.path.append('../')
import os
from transformers import *
from frameBERT.src import utils
from frameBERT.src import dataio
from frameBERT.src import eval_fn
from frameBERT import frame_parser
from frameBERT.src.modeling import BertForJointShallowSemanticParsing
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from torch import nn
from torch.optim import Adam
from tqdm import tqdm, trange

from pprint import pprint

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
if device != "cpu":
    torch.cuda.set_device(0)
import pickle

import numpy as np
import random
np.random.seed(0)   
random.seed(0)

from torch import autograd
torch.cuda.empty_cache()

import argparse

from collections import Counter

In [31]:
try:
    dir_path = os.path.dirname(os.path.abspath( __file__ ))
except:
    dir_path = '.'
    
# 실행시간 측정 함수
import time

_start_time = time.time()

def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60)
    
    result = '{}hour:{}min:{}sec'.format(t_hour,t_min,t_sec)
    return result

In [32]:
srl = 'framenet'
language = 'multilingual'
fnversion = '1.2'

In [142]:
fnversion = '1.2'
srl = 'framenet'
from koreanframenet import koreanframenet
kfn = koreanframenet.interface(version=fnversion)

# en_trn, en_dev, en_tst = dataio.load_data(srl=srl, language='en', exem=True)
en_trn, en_dev, en_tst = dataio.load_data(srl=srl, language='en', exem=False)

ekfn_trn_d, ekfn_tst_d = kfn.load_data(source='efn')
jkfn_trn_d, jkfn_tst_d = kfn.load_data(source='jfn')
skfn_trn_d, skfn_unlabel_d, skfn_tst_d = kfn.load_data(source='sejong')
pkfn_trn_d, pkfn_unlabel_d, pkfn_tst_d = kfn.load_data(source='propbank')

ekfn_trn = dataio.data2tgt_data(ekfn_trn_d, mode='train')
ekfn_tst = dataio.data2tgt_data(ekfn_tst_d, mode='train')

jkfn_trn = dataio.data2tgt_data(jkfn_trn_d, mode='train')
jkfn_tst = dataio.data2tgt_data(jkfn_tst_d, mode='train')

skfn_trn = dataio.data2tgt_data(skfn_trn_d, mode='train')
skfn_unlabel = dataio.data2tgt_data(skfn_unlabel_d, mode='train')
skfn_tst = dataio.data2tgt_data(skfn_tst_d, mode='train')

pkfn_trn = dataio.data2tgt_data(pkfn_trn_d, mode='train')
pkfn_unlabel = dataio.data2tgt_data(pkfn_unlabel_d, mode='train')
pkfn_tst = dataio.data2tgt_data(pkfn_tst_d, mode='train')

# of instances in trn: 19391
# of instances in dev: 2272
# of instances in tst: 6714
data example: [['Greece', 'wildfires', 'force', 'thousands', 'to', '<tgt>', 'evacuate', '</tgt>'], ['_', '_', '_', '_', '_', '_', 'evacuate.v', '_'], ['_', '_', '_', '_', '_', '_', 'Escaping', '_'], ['O', 'O', 'O', 'B-Escapee', 'O', 'X', 'O', 'X']]

### loading Korean FrameNet (from efn )
tuples: (trn, tst)
10647 3550

### loading Korean FrameNet (from jfn )
tuples: (trn, tst)
2200 1000

### loading Korean FrameNet (from sejong )
tuples: (trn, unlabel_data, tst)
500 4212 1000

### loading Korean FrameNet (from propbank )
tuples: (trn, unlabel_data, tst)
500 852 1000


In [34]:
print(en_trn[0])

[['Greece', 'wildfires', 'force', 'thousands', 'to', '<tgt>', 'evacuate', '</tgt>'], ['_', '_', '_', '_', '_', '_', 'evacuate.v', '_'], ['_', '_', '_', '_', '_', '_', 'Escaping', '_'], ['O', 'O', 'O', 'B-Escapee', 'O', 'X', 'O', 'X']]


In [69]:
# def get_all_tokens(data):
#     tokens = []
#     for i in data:
#         tok = i[0]
#         tokens += tok
#     return tokens

# ekfn = ekfn_trn_d
# ekfn_tst = ekfn_tst_d
# jkfn = jkfn_tst_d
# skfn = skfn_tst_d
# pkfn = pkfn_tst_d

# ekfn_tokens = get_all_tokens(ekfn)
# jkfn_tokens = get_all_tokens(jkfn)
# skfn_tokens = get_all_tokens(skfn)
# pkfn_tokens = get_all_tokens(pkfn)

# d = Counter(ekfn_tokens).most_common(50)
# print(d)
# print('')

# d = Counter(jkfn_tokens).most_common(50)
# print(d)
# print('')

# d = Counter(pkfn_tokens).most_common(50)
# print(d)
# print('')

# d = Counter(skfn_tokens).most_common(50)
# print(d)
# print('')

[('그리고', 1706), ('(', 1631), ('수', 1432), (')', 1354), ('있다.', 1288), ('위한', 1122), ('및', 1112), (':', 1102), ('위해', 1094), ('대한', 1075), ('있는', 1070), ('핵', 1031), ('더', 838), ('이', 822), ('많은', 822), ('그', 789), ('다른', 712), ('미사일', 698), ('것을', 671), ('이란은', 598), ('이란의', 572), ('한', 557), ('또한', 537), ('화학', 530), ('이란이', 523), (';', 516), ('생물', 515), ('-', 495), ('화학무기', 471), ('것이다.', 467), ('같은', 463), ('무기', 452), ('프로그램을', 447), ('미국', 413), ('가장', 398), ('모든', 395), ('것으로', 395), ('두', 369), ('나는', 367), ('탄도', 363), ('그들의', 355), ('우라늄', 352), ('새로운', 348), ('의해', 341), ('--', 336), ('그의', 335), ('핵무기', 335), ('대해', 330), ('동안', 317), ('그는', 314)]

[('수', 125), ('그', 115), ('이', 88), ('있는', 63), ('있습니다.', 53), ('것이', 48), ('것이다.', 46), ('하는', 42), ('한', 41), ('있다.', 35), ('약', 33), ('나는', 31), ('등의', 30), ('것을', 30), ('때', 30), ('있었다.', 29), ('지닌', 29), ('비행기를', 29), ('후,', 28), ('조종사', 28), ('면허,', 28), ('보고', 26), ('같은', 26), ('하지만,', 26), ('수십', 25), ('된', 25), ('하고', 24

In [80]:
d= ['O', 'O', 'O', 'B-Escapee', 'I-Escapee', 'X', 'O', 'X']

from src.dataio import frame2rdf

d = frame2rdf([ekfn_trn[2]])
print(d)

[('frame#0:Cause_to_make_progress', 'lu', '개발을'), ('frame#0:Cause_to_make_progress', 'fe:Agent', '이란은'), ('frame#0:Cause_to_make_progress', 'fe:Project', '전체 연료 주기')]


In [88]:
ekfn = ekfn_trn_d
ekfn_tst = ekfn_tst_d
jkfn = jkfn_tst_d
skfn = skfn_tst_d
pkfn = pkfn_tst_d

def get_avg_lengh(data):
    n = 0
    total = 0
    sent = 0
    for i in data:        
        sent +=1 
        graph = frame2rdf([i])        
        for s,p,o in graph:
            if 'fe:' in p:
                lengh = len(o.split(' '))                
                n += 1
                total += lengh
                
    avg_lengh = round(total / n, 2)
    avg_fe_number = round(n / sent, 2)
    
    result = 'avg_arg_lenth: ' + str(avg_lengh) + '. avg_args_per_sent: ' + str(avg_fe_number)
    return avg_lengh, avg_fe_number



print('EKFN - avg_arg_lenth:', get_avg_lengh(ekfn))
print('JKFN - avg_arg_lenth:', get_avg_lengh(jkfn))
print('SKFN - avg_arg_lenth:', get_avg_lengh(skfn))
print('PKFN - avg_arg_lenth:', get_avg_lengh(pkfn))
        

EKFN - avg_arg_lenth: (3.13, 1.29)
JKFN - avg_arg_lenth: (2.22, 1.57)
SKFN - avg_arg_lenth: (1.72, 2.13)
PKFN - avg_arg_lenth: (3.82, 2.18)


In [139]:
def get_lus_frames(data):
    lus, frames, luframes = [],[],[]
    for d in data:
        for i in range(len(d[0])):
            l = d[1][i]
            if l != '_':
                lu = l
                frame = d[2][i]
                luframe = l+'.'+frame
                lus.append(lu)
                frames.append(frame)
                luframes.append(luframe)
                break
    lus = list(set(lus))
#     frames = list(set(frames))
#     luframes = list(set(luframes))
    return lus, frames, luframes

def get_fes(data):
    fes = []
    for d in data:
        for f in d[2]:
            if f != '_':
                frame = f
        f_list = d[-1]
        for fe in f_list:
            if fe.startswith('B'):
                fe_label = fe.split('-')[1]
#                 fes.append(fe_label)
                framefe = frame + '-' + fe_label
                fes.append(framefe)
                
#     fes = list(set(fes))
    return fes

In [48]:
def get_tps(data):
    sents = len(data)
    tokens = 0
    
    for d in data:
        tokens += len(d[0])
        
    tps = round(float(tokens / sents),2)
    return tps

In [143]:
ekfn = ekfn_trn_d + ekfn_tst_d
jkfn = jkfn_trn_d + jkfn_tst_d
# skfn = skfn_trn + skfn_unlabel + skfn_tst
# pkfn = pkfn_trn + pkfn_unlabel + pkfn_tst

efn = en_trn

ekfn = ekfn_trn_d
ekfn_tst = ekfn_tst_d
jkfn = jkfn_tst_d
skfn = skfn_tst_d
pkfn = pkfn_tst_d

efn_lus, efn_frames, efn_luframes = get_lus_frames(efn)
ekfn_lus, ekfn_frames, ekfn_luframes = get_lus_frames(ekfn)
ekfn_tst_lus, ekfn_tst_frames, ekfn_tst_luframes = get_lus_frames(ekfn)
jkfn_lus, jkfn_frames, jkfn_luframes = get_lus_frames(jkfn)
skfn_lus, skfn_frames, skfn_luframes = get_lus_frames(skfn)
pkfn_lus, pkfn_frames, pkfn_luframes = get_lus_frames(pkfn)

efn_fes = get_fes(efn)
ekfn_fes = get_fes(ekfn)
ekfn_tst_fes = get_fes(ekfn_tst)
jkfn_fes = get_fes(jkfn)
skfn_fes = get_fes(skfn)
pkfn_fes = get_fes(pkfn)

In [50]:
print('# of ekfn:', len(ekfn))
print('# of jkfn:', len(jkfn))
print('# of skfn:', len(skfn))
print('# of pkfn:', len(pkfn))

# of ekfn: 10647
# of jkfn: 1000
# of skfn: 1000
# of pkfn: 1000


In [51]:
print('# of ekfn_lus:', len(ekfn_lus))
print('# of ekfn_frames:', len(ekfn_frames))
print('# of ekfn_luframes:', len(ekfn_luframes))
print('# of jkfn_lus:', len(jkfn_lus))
print('# of jkfn_frames:', len(jkfn_frames))
print('# of jkfn_luframes:', len(jkfn_luframes))
print('# of skfn_lus:', len(skfn_lus))
print('# of skfn_frames:', len(skfn_frames))
print('# of skfn_luframes:', len(skfn_luframes))
print('# of pkfn_lus:', len(pkfn_lus))
print('# of pkfn_frames:', len(pkfn_frames))
print('# of pkfn_luframes:', len(pkfn_luframes))

# of ekfn_lus: 2736
# of ekfn_frames: 610
# of ekfn_luframes: 10647
# of jkfn_lus: 649
# of jkfn_frames: 293
# of jkfn_luframes: 1000
# of skfn_lus: 835
# of skfn_frames: 337
# of skfn_luframes: 1000
# of pkfn_lus: 448
# of pkfn_frames: 282
# of pkfn_luframes: 1000


In [58]:
d = Counter(ekfn_luframes).most_common(10)
print(d)
print('')

d = Counter(jkfn_luframes).most_common(10)
print(d)
print('')
d = Counter(skfn_luframes).most_common(10)
print(d)
print('')

d = Counter(pkfn_luframes).most_common(10)
print(d)
print('')

[('프로그램.n.Project', 169), ('미사일.n.Weapon', 89), ('시설.n.Locale_by_use', 87), ('핵무기.n.Weapon', 75), ('작용제.n.Active_substance', 74), ('국가.n.Political_locales', 69), ('생산.n.Manufacturing', 68), ('말하다.v.Statement', 65), ('생산하다.v.Manufacturing', 60), ('능력.n.Capability', 59)]

[('사람.n.People', 17), ('보다.v.Perception_active', 15), ('없다.a.Existence', 10), ('있다.a.Existence', 9), ('재미있다.a.Stimulus_focus', 9), ('하다.a.Intentionally_act', 8), ('되다.a.Becoming', 8), ('도착하다.v.Arriving', 7), ('눈.n.Perception_experience', 6), ('기쁘다.a.Experiencer_focus', 6)]

[('터지다.v.Cause_to_fragment', 4), ('먹다.v.Ingestion', 4), ('채우다.v.Filling', 4), ('죽다.v.Death', 4), ('물려받다.v.Take_place_of', 3), ('빠뜨리다.v.Dunking', 3), ('내려오다.v.Traversing', 3), ('건네다.v.Giving', 3), ('올리다.v.Placing', 3), ('듣다.v.Hearsay', 3)]

[('마련하다.v.Activity_prepare', 10), ('대하다.v.Topic', 9), ('벌이다.v.Activity_start', 8), ('검토하다.v.Inspecting', 8), ('거치다.v.Undergoing', 7), ('시작하다.v.Activity_start', 7), ('발표하다.v.Expressing_publicly', 7), ('참석하다.v.Attend

In [11]:
a = set(['a','b', 'b', 'c'])
b = set(['a' ,'b'])

len(a-b)

1

In [65]:
def get_unseen(a,b):
    d = len(set(b) - set(a))
    per = '('+str(round((d / (len(b)) * 100), 2))+'%)'
    count = 100 - round((d / (len(b)) * 100), 2)
    return d, per, count

def get_text(d):
    t = ' '.join(d[0])
    return t

In [41]:
print('tokens per sentence')
print(get_tps(ekfn))
print(get_tps(jkfn))
print(get_tps(skfn))
print(get_tps(pkfn))

tokens per sentence
20.04
14.79
7.36
23.86


In [66]:
# pprint(ekfn_luframes)

In [127]:
print('unseen LU-frame by E-KFN')
print(get_unseen(ekfn_luframes, ekfn_tst_luframes))
print(get_unseen(ekfn_luframes, jkfn_luframes))
print(get_unseen(ekfn_luframes, skfn_luframes))
print(get_unseen(ekfn_luframes, pkfn_luframes))

unseen LU-frame by E-KFN
(0, '(0.0%)', 100.0)
(524, '(52.4%)', 47.6)
(723, '(72.3%)', 27.700000000000003)
(298, '(29.8%)', 70.2)


In [61]:
print('unseen LU-frame by J-KFN')
print(get_unseen(jkfn_luframes, ekfn_luframes))
print(get_unseen(jkfn_luframes, jkfn_luframes))
print(get_unseen(jkfn_luframes, skfn_luframes))
print(get_unseen(jkfn_luframes, pkfn_luframes))

unseen LU-frame by J-KFN
(3364, '(31.6%)')
(0, '(0.0%)')
(833, '(83.3%)')
(468, '(46.8%)')


In [62]:
print('unseen LU-frame by S-KFN')
print(get_unseen(skfn_luframes, ekfn_luframes))
print(get_unseen(skfn_luframes, jkfn_luframes))
print(get_unseen(skfn_luframes, skfn_luframes))
print(get_unseen(skfn_luframes, pkfn_luframes))

unseen LU-frame by S-KFN
(3410, '(32.03%)')
(680, '(68.0%)')
(0, '(0.0%)')
(396, '(39.6%)')


In [63]:
print('unseen LU-frame by P-KFN')
print(get_unseen(pkfn_luframes, ekfn_luframes))
print(get_unseen(pkfn_luframes, jkfn_luframes))
print(get_unseen(pkfn_luframes, skfn_luframes))
print(get_unseen(pkfn_luframes, pkfn_luframes))

unseen LU-frame by P-KFN
(3357, '(31.53%)')
(687, '(68.7%)')
(768, '(76.8%)')
(0, '(0.0%)')


In [136]:
print('unseen frame by E-KFN')
print(get_unseen(ekfn_frames, ekfn_frames))
print(get_unseen(ekfn_frames, jkfn_frames))
print(get_unseen(ekfn_frames, skfn_frames))
print(get_unseen(ekfn_frames, pkfn_frames))

unseen frame by E-KFN
(0, '(0.0%)', 100.0)
(31, '(3.1%)', 96.9)
(82, '(8.2%)', 91.8)
(52, '(5.2%)', 94.8)


In [116]:
print('unseen frame by EFN')
print(get_unseen(efn_frames, ekfn_frames))
print(get_unseen(efn_frames, jkfn_frames))
print(get_unseen(efn_frames, skfn_frames))
print(get_unseen(efn_frames, pkfn_frames))

unseen frame by EFN
(21, '(0.2%)', 99.8)
(17, '(1.7%)', 98.3)
(55, '(5.5%)', 94.5)
(39, '(3.9%)', 96.1)


In [20]:
print(get_text(ekfn[0]))
print(get_text(jkfn[3]))
print(get_text(skfn[0]))
print(get_text(pkfn[0]))

이란이 발전하고 있는 세계에서 가장 진보된 생명 공학 산업 중의 하나를 가지고 있으며, 백신 연구와 생산 분야에서 서남 아시아에서 리더로서 오랫동안 인식되어 왔다.
하지만 역시 다카시는 아내를 사랑하고 있었다.
나는 회사를 사직하고 시골로 들어갔다.
독일의 영국산 쇠고기 수입 금지 조치가 17 일 상원의 승인절차를 거쳐 해제될 전망이라고 바바리아주 출신의 라인홀트 보클네트 의원이 지난 15 일 밝혔다.


In [21]:


print(len(ekfn_fes))
print(len(jkfn_fes))
print(len(skfn_fes))
print(len(pkfn_fes))

2288
690
1080
880


In [135]:
print('unseen FEs by E-KFN')
print(get_unseen(ekfn_fes, ekfn_tst_fes))
print(get_unseen(ekfn_fes, jkfn_fes))
print(get_unseen(ekfn_fes, skfn_fes))
print(get_unseen(ekfn_fes, pkfn_fes))

unseen FEs by E-KFN
(292, '(6.59%)', 93.41)
(247, '(15.74%)', 84.26)
(513, '(24.1%)', 75.9)
(330, '(15.16%)', 84.84)


In [92]:
print('unseen FEs by J-KFN')
print(get_unseen(jkfn_fes, ekfn_fes))
print(get_unseen(jkfn_fes, jkfn_fes))
print(get_unseen(jkfn_fes, skfn_fes))
print(get_unseen(jkfn_fes, pkfn_fes))

unseen FEs by J-KFN
(1845, '(13.42%)', 86.58)
(0, '(0.0%)', 100.0)
(848, '(39.83%)', 60.17)
(685, '(31.47%)', 68.53)


In [93]:
print('unseen FEs by S-KFN')
print(get_unseen(skfn_fes, ekfn_fes))
print(get_unseen(skfn_fes, jkfn_fes))
print(get_unseen(skfn_fes, skfn_fes))
print(get_unseen(skfn_fes, pkfn_fes))

unseen FEs by S-KFN
(1721, '(12.52%)', 87.48)
(458, '(29.19%)', 70.81)
(0, '(0.0%)', 100.0)
(443, '(20.35%)', 79.65)


In [122]:
print('unseen FEs by P-KFN')
print(get_unseen(pkfn_fes, ekfn_fes))
print(get_unseen(pkfn_fes, jkfn_fes))
print(get_unseen(pkfn_fes, skfn_fes))
print(get_unseen(pkfn_fes, pkfn_fes))

unseen FEs by P-KFN
(357, '(61.66%)', 38.34)
(140, '(52.24%)', 47.76)
(123, '(39.94%)', 60.06)
(0, '(0.0%)', 100.0)


In [144]:
print('unseen FEs by EFN')
print(get_unseen(efn_fes, ekfn_fes))
print(get_unseen(efn_fes, jkfn_fes))
print(get_unseen(efn_fes, skfn_fes))
print(get_unseen(efn_fes, pkfn_fes))

unseen FEs by EFN
(257, '(1.87%)', 98.13)
(143, '(9.11%)', 90.89)
(387, '(18.18%)', 81.82)
(250, '(11.48%)', 88.52)


In [27]:
print(ekfn_fes)

['Ride_vehicle-Vehicle', 'Justifying-Act', 'Prohibiting_or_licensing-State', 'Locale_by_use-Descriptor', 'Purpose-Domain', 'Perception_active-Perceiver_agentive', 'Change_event_duration-Cause', 'Sounds-Sound_source', 'Moving_in_place-Fixed_location', 'Ingest_substance-Delivery_device', 'Measure_duration-Landmark_event', 'Thriving-Circumstances', 'Bringing-Goal', 'Cognitive_connection-Concept_1', 'Memory-Cognizer', 'Fame-Role', 'Frequency-Interval', 'Simple_name-Entity', 'Suasion-Speaker', 'Performers_and_roles-Performer', 'State_continue-Entity', 'Judgment_direct_address-Reason', 'Locale_by_use-Locale', 'Breathing-Agent', 'Operate_vehicle-Time', 'Natural_features-Container_possessor', 'Process_end-Frequency', 'Cause_to_make_noise-Agent', 'Cardinal_numbers-Entity', 'First_experience-Experience', 'Discussion-Interlocutor_1', 'Network-Nodes', 'Text-Title', 'Electricity-Source', 'Desiring-Focal_participant', 'Evidence-Manner', 'Event-Place', 'Manufacturing-Time', 'Seeking-Means', 'Prevaric